# Finetuning using IndoBART

In [ ]:
! git clone https://github.com/indobenchmark/indonlg.git

In [ ]:
%cd /content/indonlg
! pip install -r requirements.txt

/content/indonlg
     |████████████████████████████████| 1.5 MB 8.5 MB/s 
     |████████████████████████████████| 68 kB 8.0 MB/s 
     |████████████████████████████████| 776.8 MB 18 kB/s 
     |████████████████████████████████| 186 kB 73.8 MB/s 
     |████████████████████████████████| 3.3 MB 56.8 MB/s 
     |████████████████████████████████| 394.7 MB 19 kB/s 
     |████████████████████████████████| 1.2 MB 57.2 MB/s 
     |████████████████████████████████| 22.3 MB 1.5 MB/s 
     |████████████████████████████████| 65 kB 5.2 MB/s 
     |████████████████████████████████| 149 kB 76.5 MB/s 
     |████████████████████████████████| 1.8 MB 53.9 MB/s 
     |████████████████████████████████| 212 kB 73.7 MB/s 
     |████████████████████████████████| 136 kB 69.8 MB/s 
     |████████████████████████████████| 14.8 MB 60.4 MB/s 
     |████████████████████████████████| 3.8 MB 58.3 MB/s 
     |████████████████████████████████| 2.9 MB 54.8 MB/s 
     |████████████████████████████████| 462 kB 67.5 MB/s 
 

In [ ]:
%cd /content/indonlg/examples

/content/indonlg/examples


In [ ]:
import os, sys
sys.path.append('../')
os.chdir('../')

import torch
import shutil
import random
import numpy as np
import pandas as pd
from torch import optim
from transformers import MBartForConditionalGeneration

from indobenchmark import IndoNLGTokenizer
from utils.train_eval import train, evaluate
from utils.metrics import generation_metrics_fn
from utils.forward_fn import forward_generation
from utils.data_utils import MachineTranslationDataset, GenerationDataLoader

In [ ]:
###
# common functions
###
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
# Set random seed
# set_seed(26092020)

# Load Model

In [ ]:
bart_model = MBartForConditionalGeneration.from_pretrained('indobenchmark/indobart-v2')
tokenizer = IndoNLGTokenizer.from_pretrained('indobenchmark/indobart-v2')

model = bart_model
model

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(40004, 768, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(40004, 768, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 768, padding_idx=1)
      (layers): ModuleList(
        (0): MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=Tru

In [ ]:
count_param(model)

131543040

# Prepare Dataset

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = False
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 8
valid_batch_size = 8
test_batch_size = 8

source_lang = "[sundanese]"
target_lang = "[indonesian]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = './save/MT_SUNIBS_INZNTV/example_su_id'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:
train_dataset_path = './dataset/MT_SUNIBS_INZNTV/train_preprocess.json'
valid_dataset_path = './dataset/MT_SUNIBS_INZNTV/valid_preprocess.json'
test_dataset_path = './dataset/MT_SUNIBS_INZNTV/test_preprocess.json'

train_dataset = MachineTranslationDataset(train_dataset_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset = MachineTranslationDataset(valid_dataset_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset = MachineTranslationDataset(test_dataset_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_loader = GenerationDataLoader(dataset=train_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

# Test model to generate sequences

In [ ]:
inputs = ['aku pergi ke toko obat membeli <mask>']
bart_input = tokenizer.prepare_input_for_generation(inputs, return_tensors='pt',
                                         lang_token = '[indonesian]', decoder_lang_token='[indonesian]')

bart_input.to(device)
bart_out = model(**bart_input)
print(tokenizer.decode(bart_input['input_ids'][0]))
print(tokenizer.decode(bart_out.logits.topk(1).indices[:,:].squeeze()))

<s> aku pergi ke toko obat membeli <mask> [indonesian]
<s> aku pergi ke toko obat membeli obat.


In [ ]:
inputs = ['aku menyang pasar karo <mask>']
bart_input = tokenizer.prepare_input_for_generation(inputs, return_tensors='pt',
                                         lang_token = '[javanese]', decoder_lang_token='[javanese]')

bart_input.to(device)
bart_out = bart_model(**bart_input)
print(tokenizer.decode(bart_input['input_ids'][0]))
print(tokenizer.decode(bart_out.logits.topk(1).indices[:,:].squeeze()))

<s> aku menyang pasar karo <mask> [javanese]
<s> aku menyang pasar karo tuku </s>


In [ ]:
inputs = ['kuring ka pasar senen meuli daging <mask>']
bart_input = tokenizer.prepare_input_for_generation(inputs, return_tensors='pt',
                                         lang_token = '[sundanese]', decoder_lang_token='[sundanese]')

bart_input.to(device)
bart_out = bart_model(**bart_input)
print(tokenizer.decode(bart_input['input_ids'][0]))
print(tokenizer.decode(bart_out.logits.topk(1).indices[:,:].squeeze()))

<s> kuring ka pasar senen meuli daging <mask> [sundanese]
<s> kuring ka pasar senen meuli daging sapi.


# Test model to translate

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 150/150 [02:46<00:00,  1.11s/it]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result.csv")

== Prediction Result ==
                                                 hyp  \
0  teu terang naon - na on, tur can tangtu simkur...   
1           kabeh dala lah ar nepi ka sare ube uhna.   
2  urang mah darma dad am elan allah, anu geus di...   
3  anu hatena suci mah, sagala ge suci. sabalikna...   
4  leupas, petrus jeung yahya nep angan rerencang...   

                                               label  
0  sebab memang aku tidak sadar akan sesuatu, tet...  
1          dan mereka semuanya makan sampai kenyang.  
2  karena kita ini buatan allah, diciptakan dalam...  
3  bagi orang suci semuanya suci ; tetapi bagi or...  
4  sesudah dilepaskan pergilah petrus dan yohanes...  

== Model Performance ==
           BLEU  SacreBLEU    ROUGE1    ROUGE2    ROUGEL  ROUGELsum
count  1.000000   1.000000  1.000000  1.000000  1.000000   1.000000
mean   2.079472   2.100884  8.145745  1.953286  7.771815   7.750226
std         NaN        NaN       NaN       NaN       NaN        NaN
min    2.079

In [ ]:
metric_df

,BLEU,SacreBLEU,ROUGE1,ROUGE2,ROUGEL,ROUGELsum
0,2.079472,2.100884,8.145745,1.953286,7.771815,7.750226


# Fine Tuning & Evaluation

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:2.8207 LR:0.00010000: 100%|██████████| 746/746 [01:37<00:00,  7.69it/s]


(Epoch 1) TRAIN LOSS:2.8207 BLEU:28.94 SacreBLEU:31.31 ROUGE1:49.90 ROUGE2:20.50 ROUGEL:44.79 ROUGELsum:44.79 LR:0.00010000


VALID LOSS:2.3922: 100%|██████████| 100/100 [00:04<00:00, 24.27it/s]


(Epoch 1) VALID LOSS:2.3922 BLEU:19.86 SacreBLEU:20.18 ROUGE1:50.90 ROUGE2:21.52 ROUGEL:46.05 ROUGELsum:46.09


(Epoch 2) TRAIN LOSS:1.9553 LR:0.00009000: 100%|██████████| 746/746 [01:35<00:00,  7.79it/s]


...

(Epoch 9) VALID LOSS:2.8151 BLEU:26.05 SacreBLEU:26.39 ROUGE1:54.79 ROUGE2:27.94 ROUGEL:50.66 ROUGELsum:50.72
count stop: 1


(Epoch 10) TRAIN LOSS:0.1196 LR:0.00003874: 100%|██████████| 746/746 [01:36<00:00,  7.70it/s]


(Epoch 10) TRAIN LOSS:0.1196 BLEU:94.32 SacreBLEU:94.51 ROUGE1:97.39 ROUGE2:94.94 ROUGEL:97.36 ROUGELsum:97.36 LR:0.00003874


VALID LOSS:2.9081: 100%|██████████| 100/100 [00:04<00:00, 24.39it/s]


(Epoch 10) VALID LOSS:2.9081 BLEU:26.07 SacreBLEU:26.40 ROUGE1:54.47 ROUGE2:27.69 ROUGEL:50.40 ROUGELsum:50.40


In [ ]:
# Load best model
model.load_state_dict(torch.load(model_dir + "/best_model_0.th"))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 150/150 [01:55<00:00,  1.30it/s]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result.csv")

== Prediction Result ==
                                                 hyp  \
0  sebab aku tidak tahu apa yang harus kut ulis k...   
1          dan mereka semuanya makan sampai kenyang.   
2  sebab kami telah melakukan kehendak allah di d...   
3  untuk orang yang kudus segala sesuatu adalah k...   
4  kemudian dari pada itu petrus dan yohanes data...   

                                               label  
0  sebab memang aku tidak sadar akan sesuatu, tet...  
1          dan mereka semuanya makan sampai kenyang.  
2  karena kita ini buatan allah, diciptakan dalam...  
3  bagi orang suci semuanya suci ; tetapi bagi or...  
4  sesudah dilepaskan pergilah petrus dan yohanes...  

== Model Performance ==
            BLEU  SacreBLEU   ROUGE1    ROUGE2     ROUGEL  ROUGELsum
count   1.000000   1.000000   1.0000   1.00000   1.000000   1.000000
mean   20.954394  21.273456  46.0602  23.25705  40.774935  40.744055
std          NaN        NaN      NaN       NaN        NaN        NaN
min    2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_dir+"/best_model_0.th"

'./save/MT_SUNIBS_INZNTV/example_su_id/best_model_0.th'

In [ ]:
! cp './save/MT_SUNIBS_INZNTV/example_su_id/best_model_0.th' '/content/drive/My Drive/'